In [ ]:
IMAGES_PATH = os.path.join('data','images')

In [ ]:
import cv2
import os
import uuid
import time
import zipfile
from IPython.display import FileLink, display
from PIL import Image

# Path to store the captured images
IMAGES_PATH = 'captured_images'  # Save images in the current working directory

# Number of images you want to capture
number_images = 90  # Set this to 90 for capturing 90 images

# Initialize the video capture object to use the first camera (index 0)
cap = cv2.VideoCapture(0)

# Check if the camera is opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Create the directory to save images if it doesn't exist
os.makedirs(IMAGES_PATH, exist_ok=True)

# Loop to collect images based on the specified number of images (number_images)
for imgnum in range(number_images):
    print(f'Collecting image {imgnum}')
    
    # Capture a frame from the video feed
    ret, frame = cap.read()
    
    # Check if the frame was captured successfully
    if not ret or frame is None:
        print("Error: Failed to capture image.")
        break
    
    # Generate a unique image filename using UUID, and save the captured frame
    imgname = os.path.join(IMAGES_PATH, f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname, frame)

    # Display the captured frame in the notebook using PIL for better display
    img_display = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    display(img_display)
    
    # Add a small delay of 0.5 seconds before capturing the next image
    time.sleep(0.5)

# Release the video capture object
cap.release()

# Create a zip file of all captured images
zip_filename = 'captured_images.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for img_file in os.listdir(IMAGES_PATH):
        img_path = os.path.join(IMAGES_PATH, img_file)
        zipf.write(img_path, os.path.basename(img_path))

# Print the path to the zip file
print(f'Images saved and zipped into {zip_filename}')

# Provide a download link for the zip file in Jupyter
display(FileLink(zip_filename))


# Annotate Images with LabelMe

In [ ]:
!labelme

#  Load Image into TF Data Pipeline

In [ ]:
images = tf.data.Dataset.list_files(r'C:\Users\DELL\Downloads\data\images\*.jpg')

In [ ]:
images.as_numpy_iterator().next()

In [ ]:
def load_image(x):
    # Read the file at the path specified by 'x' into a byte string
    byte_img = tf.io.read_file(x)
    
    # Decode the byte string into a JPEG image
    # This converts the raw bytes into a usable tensor representing the image
    img = tf.io.decode_jpeg(byte_img)
    
    return img


In [ ]:
images = images.map(load_image)

In [ ]:
#retrieve the first element from a TensorFlow tf.data.Dataset as a NumPy array
images.as_numpy_iterator().next()


In [ ]:
type(images)

In [ ]:
image_generator = images.batch(4).as_numpy_iterator()
plot_images = image_generator.next()
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, image in enumerate(plot_images):
    ax[idx].imshow(image) 
plt.show()

# SPLIT DATA INTO TRAIN TEST AND VAL

In [ ]:
# 64 to train
# 17 and 17 to test and val

import os

base_path = r'C:\Users\DELL\Downloads\data'

# List images and labels in the respective folders
image_folder = os.path.join(base_path, 'images')  # All images are in the images folder
label_folder = os.path.join(base_path, 'labels')  # All labels are in the labels folder

image_files = os.listdir(image_folder)
label_files = os.listdir(label_folder)

print("Image Files:", image_files)
print("Label Files:", label_files)


In [ ]:
# Splitting 90 images into datasets:
# 90 * 0.7 = 63 images for training
# 90 * 0.15 = 14 images for testing
# 90 * 0.15 = 13 images for validation
import os
base_path = r'C:\Users\DELL\Downloads\data'

for folder in ['train', 'test', 'val']:
    image_folder = os.path.join(base_path, folder, 'images')
    label_folder = os.path.join(base_path, folder, 'labels')
    
    # Ensure the subfolders exist for both images and labels
    os.makedirs(image_folder, exist_ok=True)
    os.makedirs(label_folder, exist_ok=True)
    
    # Get all image files in the corresponding dataset folder
    for image_file in os.listdir(image_folder):
        # Extract the base filename and check for corresponding label
        image_base_name = os.path.splitext(image_file)[0]
        label_filename = image_base_name + '.json'
        existing_label_filepath = os.path.join(base_path, 'labels', label_filename)
        
        # Debugging output to verify the matching process
        print(f"Checking for label for image: {image_file}")
        
        # If the label file exists in the 'labels' folder, move it to the current folder's label folder
        if os.path.exists(existing_label_filepath):
            new_label_filepath = os.path.join(label_folder, label_filename)
            os.replace(existing_label_filepath, new_label_filepath)
            print(f"Moved label {label_filename} to {label_folder}")
        else:
            print(f"No label found for image: {image_file}")

            
#C:\Users\DELL\Downloads\data
#    ├── images               # Original images folder (unchanged)
#    ├── labels               # Original labels folder (unchanged)
#    ├── train
#    │   ├── images           # Training images
#    │   └── labels           # Training labels
#    ├── test
#    │   ├── images           # Test images
#    │   └── labels           # Test labels
#    └── val
#        ├── images           # Validation images
#        └── labels           # Validation labels
            


# Apply Image Augmentation on Images and Labels using Albumentations

In [ ]:
import albumentations as alb

# Define a data augmentation pipeline using Albumentations
augmentor = alb.Compose([
    # Randomly crop the image to 450x450 pixels
    alb.RandomCrop(width=450, height=450),  
    
    # Apply a horizontal flip with a 50% probability
    alb.HorizontalFlip(p=0.5),  
    
    # Adjust brightness and contrast with a 20% probability
    alb.RandomBrightnessContrast(p=0.2),  
    
    # Apply random gamma correction with a 20% probability
    alb.RandomGamma(p=0.2),  
    
    # Shift the RGB channels (e.g., change color tones) with a 20% probability
    alb.RGBShift(p=0.2),  
    
    # Apply a vertical flip with a 50% probability
    alb.VerticalFlip(p=0.5)  
], 
# Specify parameters for bounding box processing
bbox_params=alb.BboxParams(
    format='albumentations',  # Bounding box format is Albumentations (x_min, y_min, x_max, y_max)
    label_fields=['class_labels']  # The labels associated with each bounding box
))


### Load a Test Image and Annotation with OpenCV and JSON

In [ ]:
import os

# Define the correct folder path
label_folder_path = r'C:\Users\DELL\Downloads\data\train\labels'

# Check if the 'labels' folder exists
if os.path.exists(label_folder_path):
    print(f"Directory exists: {label_folder_path}")
else:
    print(f"Directory does not exist: {label_folder_path}")

# List the files in the directory if it exists
if os.path.exists(label_folder_path):
    label_files = os.listdir(label_folder_path)
    print("Files in labels folder:", label_files)


In [ ]:
import json
import cv2

# Define the paths to your image and label file
image_path = r'C:\Users\DELL\Downloads\data\train\images\c55065e4-b992-11ef-8e5b-c03eba00094a.jpg'
label_file_path = r'C:\Users\DELL\Downloads\data\train\labels\c55065e4-b992-11ef-8e5b-c03eba00094a.json'

# Load the image using OpenCV
img = cv2.imread(image_path)

# Load the label file
with open(label_file_path, 'r') as f:
    label = json.load(f)

# Access label points (e.g., the first shape's points)
print(label['shapes'][0]['points'])


In [ ]:
images_dir = r'C:\Users\DELL\data\train\images'  # Full path to the images directory
labels_dir = r'C:\Users\DELL\data\train\labels'  # Full path to the labels directory


###  Extract Coordinates and Rescale to Match Image Resolution

In [ ]:
# Initialize a list to store the coordinates
coords = [0, 0, 0, 0]

# Assign the x-coordinate of the first point from the label to coords[0]
coords[0] = label['shapes'][0]['points'][0][0]

# Assign the y-coordinate of the first point from the label to coords[1]
coords[1] = label['shapes'][0]['points'][0][1]

# Assign the x-coordinate of the second point from the label to coords[2]
coords[2] = label['shapes'][0]['points'][1][0]

# Assign the y-coordinate of the second point from the label to coords[3]
coords[3] = label['shapes'][0]['points'][1][1]
coords

In [ ]:
coords = list(np.divide(coords, [640,480,640,480]))
coords

### Apply Augmentations and View Results

In [ ]:
augmented = augmentor(image=img, bboxes=[coords], class_labels=['face'])

In [ ]:
print(type(img))  # Should be <class 'numpy.ndarray'>


In [ ]:
augmented['bboxes'][0][2:]

In [ ]:
augmented['bboxes']

In [ ]:
cv2.rectangle(augmented['image'], 
              tuple(np.multiply(augmented['bboxes'][0][:2], [450,450]).astype(int)),
              tuple(np.multiply(augmented['bboxes'][0][2:], [450,450]).astype(int)), 
                    (255,0,0), 2)

plt.imshow(augmented['image'])

# Build and Run Augmentation Pipeline

In [ ]:
# Define the base paths for the dataset and the augmented data
base_path = r'C:\Users\DELL\Downloads\data'  # Base path for input data (train, test, val folders)
augmented_path = r'C:\Users\DELL\Downloads\aug_data'  # Base path for output augmented data

# Loop through each data partition (train, test, val) to process images and labels
for partition in ['train', 'test', 'val']: 
    # Define paths for images and labels within the current partition
    images_path = os.path.join(base_path, partition, 'images')  # Path to the images directory
    labels_path = os.path.join(base_path, partition, 'labels')  # Path to the labels directory
    
    # Define paths to store the augmented images and labels for the current partition
    augmented_images_path = os.path.join(augmented_path, partition, 'images')  
    augmented_labels_path = os.path.join(augmented_path, partition, 'labels')  
    
    # Create directories for augmented images and labels if they do not exist
    os.makedirs(augmented_images_path, exist_ok=True)
    os.makedirs(augmented_labels_path, exist_ok=True)

    # Loop through all images in the current partition
    for image in os.listdir(images_path):
        # Load the image from the images directory
        img = cv2.imread(os.path.join(images_path, image))  

        # Default bbox coordinates in case no label file exists for the image
        coords = [0, 0, 0.00001, 0.00001]  
        
        # Define the path to the corresponding label file (JSON) for the current image
        label_path = os.path.join(labels_path, f'{image.split(".")[0]}.json')  

        # Check if the label file exists for the current image
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:  # Open and read the label file
                label = json.load(f)  # Load the JSON file into a Python dictionary

            # Extract x and y coordinates from the label points and sort them to ensure correct order
            x_coords = sorted([label['shapes'][0]['points'][0][0], label['shapes'][0]['points'][1][0]])
            y_coords = sorted([label['shapes'][0]['points'][0][1], label['shapes'][0]['points'][1][1]])
            coords = [x_coords[0], y_coords[0], x_coords[1], y_coords[1]]  # [x_min, y_min, x_max, y_max]

            # Normalize the bounding box coordinates with respect to the image dimensions
            h, w = img.shape[:2]  # Extract height (h) and width (w) of the image
            coords = list(np.divide(coords, [w, h, w, h]))  # Normalize to be relative to image size

        try: 
            # Augment each image 60 times
            for x in range(60):  
                # Apply data augmentation to the image and its corresponding bounding box
                augmented = augmentor(image=img, bboxes=[coords], class_labels=['face'])
                
                # Save the augmented image with a unique filename
                aug_img_name = f'{image.split(".")[0]}.{x}.jpg'  # Generate unique filename for each augmentation
                cv2.imwrite(os.path.join(augmented_images_path, aug_img_name), augmented['image'])  # Save image

                # Create an annotation file for the augmented image
                annotation = {}  
                annotation['image'] = aug_img_name  # Store the name of the augmented image

                # Check if the label file exists for the image and if a valid bounding box was found after augmentation
                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0:  # No bounding box was detected after augmentation
                        annotation['bbox'] = [0, 0, 0, 0]  # Default bbox if no face is detected
                        annotation['class'] = 0  # No face detected
                    else:  
                        annotation['bbox'] = augmented['bboxes'][0]  # Store the new bounding box after augmentation
                        annotation['class'] = 1  # Face detected
                else:  
                    annotation['bbox'] = [0, 0, 0, 0]  # No label file means no face detected
                    annotation['class'] = 0  # No face detected

                # Save the annotation as a JSON file corresponding to the augmented image
                aug_label_name = f'{image.split(".")[0]}.{x}.json'  # Unique filename for each annotation
                with open(os.path.join(augmented_labels_path, aug_label_name), 'w') as f:  
                    json.dump(annotation, f)  # Write the annotation to the JSON file

        except Exception as e:
            print(f"Error with image {image}: {e}")  # Print the error message if an exception occurs


### Load Augmented Images to Tensorflow Dataset

In [ ]:
import tensorflow as tf

# Paths to the image directories for train, test, and validation sets
train_images_path = r'C:\Users\DELL\Downloads\aug_data\train\images\*.jpg'
test_images_path = r'C:\Users\DELL\Downloads\aug_data\test\images\*.jpg'
val_images_path = r'C:\Users\DELL\Downloads\aug_data\val\images\*.jpg'

# Function to load and preprocess images
def load_and_preprocess_images(path):
    images = tf.data.Dataset.list_files(path, shuffle=False)  # List all image files in the specified path
    images = images.map(load_image)  # Load each image using the load_image function
    images = images.map(lambda x: tf.image.resize(x, (120, 120)))  # Resize images to 120x120 pixels
    images = images.map(lambda x: x / 255)  # Normalize image pixel values to the range [0, 1]
    return images

# Load and preprocess images for train, test, and validation datasets
train_images = load_and_preprocess_images(train_images_path)
test_images = load_and_preprocess_images(test_images_path)
val_images = load_and_preprocess_images(val_images_path)


In [ ]:
train_images.as_numpy_iterator().next()

### Prepare Labels

In [ ]:
#Build Label Loading Function
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)
        
    return [label['class']], label['bbox']

In [ ]:
import tensorflow as tf

# Paths to the image directories for train, test, and validation sets
train_images_path = r'C:\Users\DELL\Downloads\aug_data\train\images\*.jpg'
test_images_path = r'C:\Users\DELL\Downloads\aug_data\test\images\*.jpg'
val_images_path = r'C:\Users\DELL\Downloads\aug_data\val\images\*.jpg'

# Paths to the label directories for train, test, and validation sets
train_labels_path = r'C:\Users\DELL\Downloads\aug_data\train\labels\*.json'
test_labels_path = r'C:\Users\DELL\Downloads\aug_data\test\labels\*.json'
val_labels_path = r'C:\Users\DELL\Downloads\aug_data\val\labels\*.json'

# Function to load and preprocess images
def load_and_preprocess_images(path):
    images = tf.data.Dataset.list_files(path, shuffle=False)  # List all image files in the specified path
    images = images.map(load_image)  # Load each image using the load_image function
    images = images.map(lambda x: tf.image.resize(x, (120, 120)))  # Resize images to 120x120 pixels
    images = images.map(lambda x: x / 255)  # Normalize image pixel values to the range [0, 1]
    return images

# Load and preprocess images for train, test, and validation datasets
train_images = load_and_preprocess_images(train_images_path)
test_images = load_and_preprocess_images(test_images_path)
val_images = load_and_preprocess_images(val_images_path)

# Load labels to TensorFlow datasets
def load_and_preprocess_labels(path):
    labels = tf.data.Dataset.list_files(path, shuffle=False)  # List all label files in the specified path
    labels = labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))  # Load labels using the load_labels function
    return labels

# Load and preprocess labels for train, test, and validation datasets
train_labels = load_and_preprocess_labels(train_labels_path)
test_labels = load_and_preprocess_labels(test_labels_path)
val_labels = load_and_preprocess_labels(val_labels_path)

# Check first label from the train dataset
print(train_labels.as_numpy_iterator().next())


### Combine Label and Image Samples

In [ ]:

# Print the lengths of the datasets
print("Number of train images:", len(list(train_images)))
print("Number of train labels:", len(list(train_labels)))
print("Number of test images:", len(list(test_images)))
print("Number of test labels:", len(list(test_labels)))
print("Number of validation images:", len(list(val_images)))
print("Number of validation labels:", len(list(val_labels)))

# Create Final Datasets (Images/Labels)
train = tf.data.Dataset.zip((train_images, train_labels))  # Combine train images and labels into a single dataset
train = train.shuffle(5000)  # Shuffle the dataset with a buffer size of 5000
train = train.batch(8)  # Batch the dataset with a batch size of 8
train = train.prefetch(4)  # Prefetch 4 batches for optimal performance

test = tf.data.Dataset.zip((test_images, test_labels))  # Combine test images and labels into a single dataset
test = test.shuffle(1300)  # Shuffle the dataset with a buffer size of 1300
test = test.batch(8)  # Batch the dataset with a batch size of 8
test = test.prefetch(4)  # Prefetch 4 batches for optimal performance

val = tf.data.Dataset.zip((val_images, val_labels))  # Combine validation images and labels into a single dataset
val = val.shuffle(1000)  # Shuffle the dataset with a buffer size of 1000
val = val.batch(8)  # Batch the dataset with a batch size of 8
val = val.prefetch(4)  # Prefetch 4 batches for optimal performance

# Check first label from the train dataset
print(train.as_numpy_iterator().next()[1])

### View Images and Annotations

In [ ]:
# Visualize some samples from the train dataset
data_samples = train.as_numpy_iterator()  # Create an iterator for the train dataset
res = data_samples.next()  # Get the next batch of data

# Create a plot with 4 columns to visualize 4 images and their bounding boxes
fig, ax = plt.subplots(ncols=4, figsize=(20, 20))

for idx in range(4): 
    sample_image = res[0][idx].copy()  # Make a writable copy of the image
    sample_coords = res[1][1][idx]  # Extract the bounding box coordinates from the batch
    
    # Draw a rectangle around the detected face using the bounding box coordinates
    cv2.rectangle(sample_image, 
                  tuple(np.multiply(sample_coords[:2], [120, 120]).astype(int)),  # Top-left corner of the bounding box
                  tuple(np.multiply(sample_coords[2:], [120, 120]).astype(int)),  # Bottom-right corner of the bounding box
                  (255, 0, 0), 2)  # Color (red) and thickness of the rectangle

    ax[idx].imshow(sample_image)  # Display the image with the bounding box on the subplot


# Build Deep Learning using the Functional API

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

In [ ]:
vgg = VGG16(include_top=False)
vgg.summary()

###  Build instance of Network

In [ ]:
def build_model(): 
    input_layer = Input(shape=(120, 120, 3))  # Input shape of 120x120 RGB image
    
    vgg_output = VGG16(include_top=False)(input_layer)  # Use VGG16 as feature extractor

    # Classification Model  
    f1 = GlobalMaxPooling2D()(vgg_output)  # Apply global max pooling
    class1 = Dense(2048, activation='relu')(f1)  # Fully connected layer for classification
    class2 = Dense(1, activation='sigmoid')(class1)  # Output layer for binary classification (face/no face)
    
    # Bounding box model
    f2 = GlobalMaxPooling2D()(vgg_output)  # Apply global max pooling for bbox
    regress1 = Dense(2048, activation='relu')(f2)  # Fully connected layer for bounding box prediction
    regress2 = Dense(4, activation='sigmoid')(regress1)  # Output layer for bounding box (x1, y1, x2, y2)
    
    facetracker = Model(inputs=input_layer, outputs=[class2, regress2])  # Define the model with two outputs
    return facetracker


### Test out Neural Network

In [ ]:
facetracker = build_model()  # Build the model
facetracker.summary()  # Print the model summary

# Test prediction
X, y = train.as_numpy_iterator().next()  # Get a batch from the training data
X.shape  # Check the shape of input data
classes, coords = facetracker.predict(X)  # Get predictions
print(classes, coords)  # Print the predictions


### Define Losses and Optimizers

In [ ]:
# 9.1 Define Optimizer and Learning Rate (LR) Decay
batches_per_epoch = len(train)
lr_decay = (1. / 0.75 - 1) / batches_per_epoch  # Learning rate decay formula
opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=lr_decay)


# Train Neural Network

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, GlobalMaxPooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.losses import BinaryCrossentropy
import json
import glob

# Paths to the image and label directories for train, test, and validation sets
train_images_path = r'C:\Users\DELL\Downloads\aug_data\train\images\*.jpg'
test_images_path = r'C:\Users\DELL\Downloads\aug_data\test\images\*.jpg'
val_images_path = r'C:\Users\DELL\Downloads\aug_data\val\images\*.jpg'

train_labels_path = glob.glob(r'C:\Users\DELL\Downloads\aug_data\train\labels\*.json')
test_labels_path = glob.glob(r'C:\Users\DELL\Downloads\aug_data\test\labels\*.json')
val_labels_path = glob.glob(r'C:\Users\DELL\Downloads\aug_data\val\labels\*.json')

# Helper function to load and preprocess images
def load_and_preprocess_image(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (120, 120))
    image = image / 255.0
    return image

# Helper function to load labels
def load_labels(label_path):
    with open(label_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    class_label = data['class']  # Binary classification label (0 or 1)
    bbox = data['bbox']  # Bounding box coordinates [x1, y1, x2, y2]
    return class_label, bbox

# Create a TensorFlow dataset for images and labels
def create_dataset(image_paths, label_paths):
    images = [load_and_preprocess_image(img) for img in image_paths]
    labels = [load_labels(lbl) for lbl in label_paths]
    class_labels = [lbl[0] for lbl in labels]
    bboxes = [lbl[1] for lbl in labels]
    dataset = tf.data.Dataset.from_tensor_slices((images, (class_labels, bboxes)))
    dataset = dataset.shuffle(500).batch(8).prefetch(tf.data.AUTOTUNE)
    return dataset

# Prepare datasets
train_image_paths = glob.glob(train_images_path)
test_image_paths = glob.glob(test_images_path)
val_image_paths = glob.glob(val_images_path)

train_dataset = create_dataset(train_image_paths, train_labels_path)
test_dataset = create_dataset(test_image_paths, test_labels_path)
val_dataset = create_dataset(val_image_paths, val_labels_path)

# Build the model
def build_model():
    input_layer = Input(shape=(120, 120, 3))
    base_model = VGG16(include_top=False, input_tensor=input_layer)
    
    # Classification head
    x = GlobalMaxPooling2D()(base_model.output)
    class_head = Dense(2048, activation='relu')(x)
    class_output = Dense(1, activation='sigmoid', name='class_output')(class_head)
    
    # Regression head
    y = GlobalMaxPooling2D()(base_model.output)
    bbox_head = Dense(2048, activation='relu')(y)
    bbox_output = Dense(4, activation='sigmoid', name='bbox_output')(bbox_head)
    
    model = Model(inputs=input_layer, outputs=[class_output, bbox_output])
    return model

facetracker = build_model()

# Custom loss functions
def localization_loss(y_true, y_pred):
    delta_coord = tf.reduce_sum(tf.square(y_true[:, :2] - y_pred[:, :2]))
    h_true = y_true[:, 3] - y_true[:, 1]
    w_true = y_true[:, 2] - y_true[:, 0]
    h_pred = y_pred[:, 3] - y_pred[:, 1]
    w_pred = y_pred[:, 2] - y_pred[:, 0]
    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true - h_pred))
    return delta_coord + delta_size

class_loss = BinaryCrossentropy()

# Compile the model
facetracker.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss={'class_output': class_loss, 'bbox_output': localization_loss},
    loss_weights={'class_output': 0.5, 'bbox_output': 1.0},
    metrics={'class_output': 'accuracy'}
)

# Train the model
facetracker.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)


# Make Predictions

In [ ]:
def rescale_bboxes(coords, original_shape):
    height, width = original_shape[:2]
    x1, y1, x2, y2 = coords
    return int(x1 * width), int(y1 * height), int(x2 * width), int(y2 * height)


In [ ]:
# Function to make predictions on a single image
def predict_and_draw(image_path, model):
    # Load and preprocess the image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    original_shape = image.shape  # Store the original shape for scaling
    image_resized = tf.image.resize(image, (120, 120)) / 255.0
    image_resized = tf.expand_dims(image_resized, axis=0)  # Add batch dimension

    # Get predictions from the model
    class_pred, bbox_pred = model.predict(image_resized)
    class_pred = class_pred[0][0]  # Extract the first batch prediction
    bbox_pred = bbox_pred[0]  # Extract the first batch bounding box

    # Rescale bounding box to original image size
    x1, y1, x2, y2 = rescale_bboxes(bbox_pred, original_shape)

    # Draw bounding box on the original image
    image_with_bbox = cv2.cvtColor(image.numpy(), cv2.COLOR_RGB2BGR)
    if class_pred > 0.5:  # If class confidence is high enough
        cv2.rectangle(image_with_bbox, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw a green box

    # Plot the image with bounding box
    plt.imshow(cv2.cvtColor(image_with_bbox, cv2.COLOR_BGR2RGB))
    plt.title(f"Face: {'Yes' if class_pred > 0.5 else 'No'} | Confidence: {class_pred:.2f}")
    plt.axis('off')
    plt.show()


In [ ]:
test_image_path = r"C:\Users\DELL\Downloads\WhatsApp Image 2024-12-15 at 21.38.11_934a5c03.jpg"
predict_and_draw(test_image_path, facetracker)


In [ ]:
#"C:\Users\DELL\Downloads\WhatsApp Image 2024-11-20 at 20.27.07_5bfadcd2.jpg"
test_image_path = r"C:\Users\DELL\Downloads\WhatsApp Image 2024-12-15 at 21.44.58_9a98589c.jpg"
predict_and_draw(test_image_path, facetracker)


### Save the Model

In [ ]:
from tensorflow.keras.models import load_model
facetracker.save('facetracker.h5')
facetracker = load_model('facetracker.h5')

# Real Time Detection

In [ ]:
import cv2
print(cv2.__version__)

In [ ]:
import os
os.environ['OPENCV_WINDOW'] = 'Qt'  # or 'GTK'

In [ ]:
import cv2
print(cv2.getBuildInformation())


In [ ]:
# Get the current working directory
current_dir = os.getcwd()

# Modify the path based on your model location
model_path = os.path.join(current_dir, 'models', 'your_model.h5')  # Adjust the path

# Print the model path for verification
print(f"Loading model from: {model_path}")

# ... (rest of your code)

# Function to preprocess and make predictions
def predict_and_draw(frame):
    # Preprocess the frame
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_resized = tf.image.resize(image, (120, 120)) / 255.0
    image_resized = tf.expand_dims(image_resized, axis=0)

    # Make predictions
    yhat = model.predict(image_resized)
    class_pred = yhat[0][0]
    bbox_pred = yhat[1][0]

    # Rescale bounding box to original frame size
    x1, y1, x2, y2 = rescale_bboxes(bbox_pred, frame.shape[:2])

    # Draw bounding box on the frame
    if class_pred > 0.5:
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    return frame

# Start the video capture
cap = cv2.VideoCapture(0)  # 0 for default camera

while True:
    ret, frame = cap.read()

    if not ret:
        print("Error: Unable to capture frame from webcam")
        break  # Exit the loop if there's an error

    # Process the frame (preprocessing, prediction, drawing)
    frame = predict_and_draw(frame)

    # Display the frame using Matplotlib
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
    plt.clf()  # Clear the plot

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()